In [41]:
from osgeo import gdal,ogr,osr
import math

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def GetExtent(gt,cols,rows):
    ''' Return list of corner coordinates from a geotransform

        @type gt:   C{tuple/list}
        @param gt: geotransform
        @type cols:   C{int}
        @param cols: number of columns in the dataset
        @type rows:   C{int}
        @param rows: number of rows in the dataset
        @rtype:    C{[float,...,float]}
        @return:   coordinates of each corner
    '''
    ext=[]
    xarr=[0,cols]
    yarr=[0,rows]

    for px in xarr:
        for py in yarr:
            x=gt[0]+(px*gt[1])+(py*gt[2])
            y=gt[3]+(px*gt[4])+(py*gt[5])
            x = round_up(x, 3)
            y = round_up(y, 3)
            ext.append([x,y])
            #print(x,y)
        yarr.reverse()
    return ext

def ReprojectCoords(coords,src_srs,tgt_srs):
    ''' Reproject a list of x,y coordinates.

        @type geom:     C{tuple/list}
        @param geom:    List of [[x,y],...[x,y]] coordinates
        @type src_srs:  C{osr.SpatialReference}
        @param src_srs: OSR SpatialReference object
        @type tgt_srs:  C{osr.SpatialReference}
        @param tgt_srs: OSR SpatialReference object
        @rtype:         C{tuple/list}
        @return:        List of transformed [[x,y],...[x,y]] coordinates
    '''
    trans_coords=[]
    transform = osr.CoordinateTransformation( src_srs, tgt_srs)
    for x,y in coords:
        x,y,z = transform.TransformPoint(x,y)
        trans_coords.append([x,y])
    return trans_coords

def rasterExtent(file_path):
    raster=file_path
    ds=gdal.Open(raster)

    gt=ds.GetGeoTransform()
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    ext=GetExtent(gt,cols,rows)
    return ext

#ext1 = rasterExtent('/home/lois/Desktop/h_tile_3600_82800.tif')
#ext2 = rasterExtent('/home/lois/Desktop/tile_3600_82800.tif')
#print(ext1==ext2)
#src_srs=osr.SpatialReference()
#src_srs.ImportFromWkt(ds.GetProjection())
#tgt_srs=osr.SpatialReference()
#tgt_srs.ImportFromEPSG(4326)
#tgt_srs = src_srs.CloneGeogCS()

#geo_ext=ReprojectCoords(ext,src_srs,tgt_srs)


In [43]:
import os
path = '/media/lois/Seagate Bac/road_building_extraction/tiled outputs'

ext1 = []
ext2 = []
for filename in sorted(os.listdir(path+'/tiled hillshade')):
    ext1.append(rasterExtent(path +'/tiled hillshade/' + filename))
    ext2.append(rasterExtent(path +'/tiled gmaps2/' + filename))

ext1 == ext2

True